In [3]:
# 1. Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from numpy import array

In [4]:
# 2. Đọc file csv và gắng index với giá Close
df = pd.read_csv('D:/UIT/DA_Gr07/Dataset/ADBE_Stock.csv')
df1=df.reset_index()['Close']

In [5]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [6]:
# 4. Chia train test
train_size = int(0.8 * len(df1))
test_size = len(df1) - train_size

train_data = df1[:train_size]
test_data = df1[train_size:]

In [7]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [8]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [9]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
if len(X_train.shape) == 2:
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
if len(X_test.shape) == 2:
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [10]:
from tensorflow.keras.layers import RNN, SimpleRNNCell
from tensorflow.keras.initializers import glorot_uniform

class AdditiveRNNCell(SimpleRNNCell):
    def build(self, input_shape):
        super().build(input_shape)
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer=glorot_uniform(),
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units),
                                                initializer=glorot_uniform(),
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units,),
                                    initializer='zeros',
                                    name='bias')
        self.built = True

    def __call__(self, inputs, states, training=None):
        prev_output = states[0]
        h = prev_output + self.activation(tf.matmul(inputs, self.kernel) + tf.matmul(prev_output, self.recurrent_kernel) + self.bias)
        return h, [h]

In [11]:
# Xây dựng model AddRNN
inputs = Input(shape=(time_step, 1))
x = RNN(AdditiveRNNCell(50), return_sequences=False)(inputs)  # 50 units
outputs = Dense(1)(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (RNN)                       │ (None, 50)             │         5,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,251 (20.51 KB)

 Trainable params: 5,251 (20.51 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 9. Fit mô hình với dữ liệu train
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'recurrent_kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 1524.6820 - val_loss: 269.2218
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 568.4610 - val_loss: 134.1254
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 247.7333 - val_loss: 32.8034
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 172.3113 - val_loss: 68.1607
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 92.2862 - val_loss: 3.8354
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 86.2390 - val_loss: 4.0590
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 52.7379 - val_loss: 2.1362
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 80.5076 - val_loss: 234.1218
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 669.7822 - val_loss: 62.2124
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 253.0304 - val_loss: 39.1711
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 168.2133 - val_loss: 11.1073
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━

In [13]:
# 10. Dự báo dữ liệu test, val
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [14]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)

In [28]:
# 12. Đánh giá độ chính xác thuật toán bằng RMSE
ytest_original = scaler.inverse_transform(ytest.reshape(-1, 1))
test_rmse = np.sqrt(np.mean((y_pred - ytest_original)**2))
print('Testing RMSE:', test_rmse)

Testing RMSE: 239875.84281744872


In [24]:
# 13. Đánh giá độ chính xác thuật toán bằng MAE
def MAE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

# Transform yval and ytest back to original scale
ytest_original = scaler.inverse_transform(ytest.reshape(-1, 1))

# Calculate MAE for validation and test data
test_mae = MAE(ytest_original, y_pred)
print('Testing MAE:', test_mae)

Testing MAE: 238750.64627048792


In [25]:
# 13. Đánh giá độ chính xác thuật toán bằng MAPE
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Transform yval and ytest back to original scale
ytest_original = scaler.inverse_transform(ytest.reshape(-1, 1))

# Calculate MAPE for validation and test data
test_mape = MAPE(ytest_original, y_pred)

print('Testing MAPE:', test_mape)

Testing MAPE: 99.85541271286549


In [18]:
# Dự báo 30 ngày tiếp theo
x_input = test_data[-time_step:].reshape(1, -1)
temp_input = list(x_input[0])

lst30_output = []
n_steps = time_step
i = 0
while i < 30:
    if len(temp_input) > n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))
    else:
        x_input = np.array(temp_input).reshape((1, len(temp_input), 1))

    yhat = model.predict(x_input, verbose=0)
    temp_input.extend(yhat[0].tolist())
    lst30_output.extend(yhat.tolist())
    i += 1

# Chuyển đổi giá trị dự đoán về phạm vi ban đầu
lst30_output = scaler.inverse_transform(lst30_output)

# In giá trị dự đoán
print("Dự báo 30 ngày tiếp theo:", lst30_output)

Dự báo 30 ngày tiếp theo: [[  83.19416331]
 [  43.34792627]
 [  13.71862669]
 [   9.26584138]
 [  47.01442649]
 [  34.64172155]
 [ -78.82204785]
 [  -9.6795416 ]
 [ 192.14268321]
 [ 319.94750014]
 [ 401.00014547]
 [ 527.17918749]
 [ 538.04993452]
 [ 624.62260665]
 [ 741.86177993]
 [ 715.15028902]
 [ 605.38376409]
 [ 579.80023424]
 [ 782.28523084]
 [ 919.26753504]
 [ 982.84379473]
 [ 917.91091122]
 [ 879.85886397]
 [ 714.89425496]
 [ 653.43222764]
 [ 949.49811145]
 [ 344.86403232]
 [ -13.21051643]
 [-275.09624132]
 [ -63.93482794]]


In [26]:
# Dự báo 60 ngày tiếp theo
x_input = test_data[-time_step:].reshape(1, -1)
temp_input = list(x_input[0])

lst60_output = []
n_steps = time_step
i = 0
while i < 60:
    if len(temp_input) > n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))
    else:
        x_input = np.array(temp_input).reshape((1, len(temp_input), 1))

    yhat = model.predict(x_input, verbose=0)
    temp_input.extend(yhat[0].tolist())
    lst60_output.extend(yhat.tolist())
    i += 1

# Chuyển đổi giá trị dự đoán về phạm vi ban đầu
lst60_output = scaler.inverse_transform(lst60_output)

# In giá trị dự đoán
print("Dự báo 60 ngày tiếp theo:", lst60_output)

Dự báo 60 ngày tiếp theo: [[  83.19416331]
 [  43.34792627]
 [  13.71862669]
 [   9.26584138]
 [  47.01442649]
 [  34.64172155]
 [ -78.82204785]
 [  -9.6795416 ]
 [ 192.14268321]
 [ 319.94750014]
 [ 401.00014547]
 [ 527.17918749]
 [ 538.04993452]
 [ 624.62260665]
 [ 741.86177993]
 [ 715.15028902]
 [ 605.38376409]
 [ 579.80023424]
 [ 782.28523084]
 [ 919.26753504]
 [ 982.84379473]
 [ 917.91091122]
 [ 879.85886397]
 [ 714.89425496]
 [ 653.43222764]
 [ 949.49811145]
 [ 344.86403232]
 [ -13.21051643]
 [-275.09624132]
 [ -63.93482794]
 [ 398.892491  ]
 [ 330.98808361]
 [ -17.91969764]
 [ 228.48795629]
 [ 594.71883164]
 [ 683.46931819]
 [ 650.08075579]
 [ -40.50538556]
 [-367.23185169]
 [-336.06067449]
 [-301.14074102]
 [ 515.28825873]
 [ 831.56902124]
 [ 760.22717055]
 [ 706.08142108]
 [ -94.83901222]
 [ 702.91214024]
 [ 994.64963727]
 [1091.44229674]
 [ 794.53785358]
 [ 554.08501719]
 [ 833.19667001]
 [ 854.17879797]
 [ 835.28851943]
 [ 949.37337691]
 [1018.78910505]
 [1010.44600294]
 [ 83

In [27]:
# Dự báo 90 ngày tiếp theo
x_input = test_data[-time_step:].reshape(1, -1)
temp_input = list(x_input[0])

lst90_output = []
n_steps = time_step
i = 0
while i < 90:
    if len(temp_input) > n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))
    else:
        x_input = np.array(temp_input).reshape((1, len(temp_input), 1))

    yhat = model.predict(x_input, verbose=0)
    temp_input.extend(yhat[0].tolist())
    lst90_output.extend(yhat.tolist())
    i += 1

# Chuyển đổi giá trị dự đoán về phạm vi ban đầu
lst90_output = scaler.inverse_transform(lst90_output)

# In giá trị dự đoán
print("Dự báo 90 ngày tiếp theo:", lst90_output)

Dự báo 90 ngày tiếp theo: [[  83.19416331]
 [  43.34792627]
 [  13.71862669]
 [   9.26584138]
 [  47.01442649]
 [  34.64172155]
 [ -78.82204785]
 [  -9.6795416 ]
 [ 192.14268321]
 [ 319.94750014]
 [ 401.00014547]
 [ 527.17918749]
 [ 538.04993452]
 [ 624.62260665]
 [ 741.86177993]
 [ 715.15028902]
 [ 605.38376409]
 [ 579.80023424]
 [ 782.28523084]
 [ 919.26753504]
 [ 982.84379473]
 [ 917.91091122]
 [ 879.85886397]
 [ 714.89425496]
 [ 653.43222764]
 [ 949.49811145]
 [ 344.86403232]
 [ -13.21051643]
 [-275.09624132]
 [ -63.93482794]
 [ 398.892491  ]
 [ 330.98808361]
 [ -17.91969764]
 [ 228.48795629]
 [ 594.71883164]
 [ 683.46931819]
 [ 650.08075579]
 [ -40.50538556]
 [-367.23185169]
 [-336.06067449]
 [-301.14074102]
 [ 515.28825873]
 [ 831.56902124]
 [ 760.22717055]
 [ 706.08142108]
 [ -94.83901222]
 [ 702.91214024]
 [ 994.64963727]
 [1091.44229674]
 [ 794.53785358]
 [ 554.08501719]
 [ 833.19667001]
 [ 854.17879797]
 [ 835.28851943]
 [ 949.37337691]
 [1018.78910505]
 [1010.44600294]
 [ 83

In [30]:
# 14.Vẽ hình
# plt.plot(pd.index, scaler.inverse_transform(train_data), label='Train')
# plt.plot(pd.index[:train_size - time_step -1], train_predict, label='Train Predict')
# plt.plot(pd.index[train_size + time_step: -1], y_pred, label='Test Predict')
# plt.legend()
# plt.show()


# train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
# plt.plot(scaler.inverse_transform(train_data))

test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))

test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))

prediect30_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1) +29, step=1)
plt.plot(prediect30_data_index,(lst30_output))

prediect60_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+59, step=1)
plt.plot(prediect60_data_index,scaler.inverse_transform(lst60_output))

prediect90_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+89, step=1)
plt.plot(prediect90_data_index,scaler.inverse_transform(lst90_output))

plt.legend(['Test','Predict', 'Predict30', 'Predict60','Predict90'])
plt.show()

In [22]:
prediect30_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1) +29, step=1)
plt.plot(prediect30_data_index,(lst30_output))

prediect60_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+59, step=1)
plt.plot(prediect60_data_index,scaler.inverse_transform(lst60_output))

prediect90_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+89, step=1)
plt.plot(prediect90_data_index,scaler.inverse_transform(lst90_output))

plt.legend(['Predict30', 'Predict60','Predict90'])
plt.show()